In [ ]:
import numpy as np
import torch
import pychop
from numpy import linalg
import jax
# from pychop.chop import chop
# from pychop.quant import quant
from time import time
np.random.seed(0)

In [2]:
X_np = np.random.randn(500, 500) # Numpy array
X_th = torch.Tensor(X_np) # torch array
X_jx = jax.numpy.asarray(X_np)
print(X_np)

[[ 1.76405235  0.40015721  0.97873798 ... -0.51423397 -1.01804188
  -0.07785476]
 [ 0.38273243 -0.03424228  1.09634685 ...  0.0941923  -1.14761094
  -0.35811408]
 [ 0.55596268  0.89247389 -0.42231482 ... -0.24679356 -1.07934317
  -0.11422555]
 ...
 [-1.28277909 -0.17950901  0.27819164 ... -1.31869965 -0.22116722
   0.41593528]
 [-1.23162889 -0.51221521  0.12511042 ...  0.84961987 -0.81843243
   0.16746637]
 [-1.09316138 -0.74909675  1.11398219 ... -0.52799973 -0.65857377
  -0.55711525]]


### print unit-roundoff in machine

In [3]:
from pychop import float_params

float_params()

,,u,xmins,xmin,xmax,p,emins,emin,emax
0,q43,6.25e-02,1.95e-03,1.56e-02,2.40e+02,4,-9,-6,7
1,q52,1.25e-01,1.53e-05,6.10e-05,5.73e+04,3,-16,-14,15
2,b,3.91e-03,9.18e-41,1.18e-38,3.39e+38,8,-133,-126,127
3,h,4.88e-04,5.96e-08,6.10e-05,6.55e+04,11,-24,-14,15
4,t,4.88e-04,1.15e-41,1.18e-38,3.40e+38,11,-136,-126,127
5,s,5.96e-08,1.40e-45,1.18e-38,3.40e+38,24,-149,-126,127
6,d,1.11e-16,4.94e-324,2.23e-308,1.80e+308,53,-1074,-1022,1023
7,q,9.63e-35,0.00e+00,0.00e+00,inf,113,-16494,-16382,16383


### set backend

In [4]:
pychop.backend('torch')

In [5]:
pychop.backend('numpy', 1) # print information, NumPy is the default option.

Load NumPy backend.


### run chop

In [6]:
from pychop.chop import chop
pyq_f = chop('h')
pyq_f(X_np)

array([[ 1.76367188,  0.40014648,  0.97851562, ..., -0.51416016,
        -1.01757812, -0.07788086],
       [ 0.3828125 , -0.03424072,  1.09667969, ...,  0.09417725,
        -1.14746094, -0.3581543 ],
       [ 0.55615234,  0.89257812, -0.42236328, ..., -0.24682617,
        -1.07910156, -0.11419678],
       ...,
       [-1.28320312, -0.17956543,  0.27807617, ..., -1.31835938,
        -0.22119141,  0.41601562],
       [-1.23144531, -0.51220703,  0.12512207, ...,  0.84960938,
        -0.81835938,  0.16748047],
       [-1.09277344, -0.74902344,  1.11425781, ..., -0.52783203,
        -0.65869141, -0.55712891]])

In [7]:
print("2^"+str(int(np.log2(pyq_f.u))))

2^-11


In [8]:
pychop.backend('torch', 1) # print information
pyq_f = chop('h')
pyq_f(X_th)

Load Troch backend.


tensor([[ 1.7637,  0.4001,  0.9785,  ..., -0.5142, -1.0176, -0.0779],
        [ 0.3828, -0.0342,  1.0967,  ...,  0.0942, -1.1475, -0.3582],
        [ 0.5562,  0.8926, -0.4224,  ..., -0.2468, -1.0791, -0.1142],
        ...,
        [-1.2832, -0.1796,  0.2781,  ..., -1.3184, -0.2212,  0.4160],
        [-1.2314, -0.5122,  0.1251,  ...,  0.8496, -0.8184,  0.1675],
        [-1.0928, -0.7490,  1.1143,  ..., -0.5278, -0.6587, -0.5571]])

In [9]:
pychop.backend('torch')
pyq_f = chop('h', device='cuda')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_gpu = X_th.to(device)
pyq_f(X_gpu)

tensor([[ 1.7637,  0.4001,  0.9785,  ..., -0.5142, -1.0176, -0.0779],
        [ 0.3828, -0.0342,  1.0967,  ...,  0.0942, -1.1475, -0.3582],
        [ 0.5562,  0.8926, -0.4224,  ..., -0.2468, -1.0791, -0.1142],
        ...,
        [-1.2832, -0.1796,  0.2781,  ..., -1.3184, -0.2212,  0.4160],
        [-1.2314, -0.5122,  0.1251,  ...,  0.8496, -0.8184,  0.1675],
        [-1.0928, -0.7490,  1.1143,  ..., -0.5278, -0.6587, -0.5571]],
       device='cuda:0')

In [10]:
pychop.backend('jax', 1) # print information
pyq_f = chop('h')
pyq_f(X_jx)

Load JAX backend.


Array([[ 1.7636719 ,  0.40014648,  0.9785156 , ..., -0.51416016,
        -1.0175781 , -0.07788086],
       [ 0.3828125 , -0.03424072,  1.0966797 , ...,  0.09417725,
        -1.1474609 , -0.3581543 ],
       [ 0.55615234,  0.8925781 , -0.42236328, ..., -0.24682617,
        -1.0791016 , -0.11419678],
       ...,
       [-1.2832031 , -0.17956543,  0.27807617, ..., -1.3183594 ,
        -0.2211914 ,  0.41601562],
       [-1.2314453 , -0.51220703,  0.12512207, ...,  0.8496094 ,
        -0.8183594 ,  0.16748047],
       [-1.0927734 , -0.74902344,  1.1142578 , ..., -0.52783203,
        -0.6586914 , -0.5571289 ]], dtype=float32)

In [11]:
from pychop.chop import chop
pychop.backend('numpy')
pyq_f = chop('h', rmode=5, flip=0)
pyq_f(X_np)

array([[ 1.76367188,  0.40014648,  0.97851562, ..., -0.51416016,
        -1.01855469, -0.07788086],
       [ 0.3828125 , -0.03424072,  1.09667969, ...,  0.09417725,
        -1.14746094, -0.3581543 ],
       [ 0.55615234,  0.89257812, -0.42236328, ..., -0.24682617,
        -1.08007812, -0.11425781],
       ...,
       [-1.28222656, -0.17944336,  0.27807617, ..., -1.31933594,
        -0.22119141,  0.41577148],
       [-1.23242188, -0.51220703,  0.12512207, ...,  0.84960938,
        -0.81835938,  0.16748047],
       [-1.09277344, -0.74902344,  1.11425781, ..., -0.52832031,
        -0.65869141, -0.55712891]])

### integer quantization

In [12]:
pychop.backend('numpy')
pyq_f = pychop.quant(bits=8)
X_q = pyq_f(X_np)
X_inv = pyq_f.dequant(X_q)
linalg.norm(X_inv - X_np)

5.165178960488854

In [13]:
pychop.backend('torch')
pyq_f = pychop.quant(bits=8)
X_q = pyq_f(X_th)
X_inv = pyq_f.dequant(X_q)
linalg.norm(X_inv - X_np)

5.165179458809817

In [14]:
pychop.backend('jax')
pyq_f = pychop.quant(bits=8)
X_q = pyq_f(X_jx)
X_inv = pyq_f.dequant(X_q)
linalg.norm(X_inv - X_jx)

5.1651783

### fixed point quantization

In [15]:
pychop.backend('numpy')
pyq_f = pychop.fpoint()

pyq_f(X_np)

array([[ 1.75  ,  0.375 ,  1.    , ..., -0.5   , -1.    , -0.0625],
       [ 0.375 , -0.0625,  1.125 , ...,  0.125 , -1.125 , -0.375 ],
       [ 0.5625,  0.875 , -0.4375, ..., -0.25  , -1.0625, -0.125 ],
       ...,
       [-1.3125, -0.1875,  0.25  , ..., -1.3125, -0.25  ,  0.4375],
       [-1.25  , -0.5   ,  0.125 , ...,  0.875 , -0.8125,  0.1875],
       [-1.0625, -0.75  ,  1.125 , ..., -0.5   , -0.6875, -0.5625]])

In [16]:
pychop.backend('torch')
pyq_f = pychop.fpoint()
pyq_f(X_th)

tensor([[ 1.7500,  0.3750,  1.0000,  ..., -0.5000, -1.0000, -0.0625],
        [ 0.3750, -0.0625,  1.1250,  ...,  0.1250, -1.1250, -0.3750],
        [ 0.5625,  0.8750, -0.4375,  ..., -0.2500, -1.0625, -0.1250],
        ...,
        [-1.3125, -0.1875,  0.2500,  ..., -1.3125, -0.2500,  0.4375],
        [-1.2500, -0.5000,  0.1250,  ...,  0.8750, -0.8125,  0.1875],
        [-1.0625, -0.7500,  1.1250,  ..., -0.5000, -0.6875, -0.5625]])

In [17]:
pychop.backend('jax')
pyq_f = pychop.fpoint()
pyq_f(X_jx)

Array([[ 1.75  ,  0.375 ,  1.    , ..., -0.5   , -1.    , -0.0625],
       [ 0.375 , -0.0625,  1.125 , ...,  0.125 , -1.125 , -0.375 ],
       [ 0.5625,  0.875 , -0.4375, ..., -0.25  , -1.0625, -0.125 ],
       ...,
       [-1.3125, -0.1875,  0.25  , ..., -1.3125, -0.25  ,  0.4375],
       [-1.25  , -0.5   ,  0.125 , ...,  0.875 , -0.8125,  0.1875],
       [-1.0625, -0.75  ,  1.125 , ..., -0.5   , -0.6875, -0.5625]],      dtype=float32)